# Introduction
The largest challenge facing the usage of metagenomic approaches in microbiology is the need to extend traditional microbiology training to include metagenomic or sequencing data analysis. Sean Eddy (a compuational biologist at the Howard Hughes Medical Institute) nicely describes the impacts of high throughput sequencing on biology and its training in his keynote address (http://cryptogenomicon.org/2014/11/01/high-throughput-sequencing-for-neuroscience/#more-858).

To facilitate the barriers to microbiologists for metagenomic assembly, we have complemented this review with a tutorial of how to estimate the abundance of reference genes in a metagenome.  We include approaches that include using references that are both (i) available genome references or (ii) assembled from the metagenome. In general, to complete this tutorial and most metagenomic assembly, one would need:

* Access to a server.  Most metagenomic assembly will require more memory than most researchers will have on their personal computers.  In this tutorial, we will provide training on the publicly accessible Amazon EC2 instances which can be rented by anyone with a registered account.
* Access to a metagenomic dataset.  We have selected the usage of the HMP Mock Community WGS dataset (http://www.hmpdacc.org/HMMC/) for this tutorial given its availability, practical size, and availability of reference genomes.  This dataset represents a synthetic metagenome of 21 known organisms for which DNA was extracted from cultured isolates, combined, and sequenced.
* Software for assembly, read mapping, and annotation.  We will demonstrate the installation of this software on an Ubuntu-based server.


# How to use this IPython Notebook

IPython notebooks are very useful to collaboratively train bioinformatics.  These notebooks have recently been featured in Nature News (http://www.nature.com/news/interactive-notebooks-sharing-the-code-1.16261 and   http://www.nature.com/news/programming-pick-up-python-1.16833)

In using these notebooks, there are a few imporant things to note.  There are two types of content in this tutorial:  text and code.  This content is placed in this notebook as "cells". If you click around on this page, you'll see different cells highlighted. To execute each cell (regardless of content), you hit on your keyboard *SHIFT+ENTER*.  If the cell contains text, the content will be displayed directly.  If the cell contains code, the code will then execute.  Also, you can execute all cells in the notebook by going to the *Cell* tab where File, Edit, View, Insert, Cell... are in the top left of this webpage and selecting *Run all*.


## 1.  Start up an EC2 Instance
Instructions for starting up an EC2 server can be found C. Titus Brown's led Next-Gen Sequencing Analysis tutorials:  http://angus.readthedocs.org/en/2014/amazon/start-up-an-ec2-instance.html.  

## 2.  Download the tutorial dataset.

We will begin this tutorial by downloading the HMP mock metagenome from the NCBI Short Read Archives (SRA).  Many public metagenomes are stored as SRA files in the NCBI. The easiest way to get these SRA files is to use a special set of tools called the *sratoolkit*.  If you have your dataset SRA run ID (in this case SRR172903), you can download the dataset and convert it to the standard "fasta" or "fastq" sequencing format is to use a special program to convert the file.  





The command below downloads the dataset in "fastq" format into the same directory where this notebook is saved.

In [1]:
!sratoolkit.2.4.5-2-ubuntu64/bin/fastq-dump SRR172903 

/bin/sh: wget: command not found


### Checking the diversity - What is the distribution of "Who is There?"

An advantage of metagenomic sequencing is the ability to quantify microbial diversity in an environment without the need to first cultivate cells.  Typically, most studies access taxonomic diversity (especially with the usage of targeted sequencing of the 16S rRNA gene*).  Diversity can also be measured in the representation of specific sequence patterns in a metagenome.  For example, one can quantify the abundance of unique nucleotide "words" of length K, or k-mers, in a metagenome.  These k-mers are also used in the assembly of metagenomes where overlapping k-mers are indicative of reads that should be connected together.  The diversity of these k-mers can give you insight into the the diversity of your sample.  Further, since assembly compares each k-mersagainst all k-mers, increasing k-mers present will require more computational memory. A nice review on k-mers and assembly is Miller et al.

(*Note that 16S rRNA amplicon sequencing is a targeted approach and not considered metagenomics.  Metagenomic sequencing uses DNA extracted from all cells in a community and sequenced.  Targeted sequencing amplifies a specific genomic locus and independently sequenced.  A great review on metagenome analysis is Sharpton et al.)

The following script is a script contained within the *khmer* package (www.github.com/ged/khmer) and can estimate the total unique number of k-mers in your dataset.  Use cases for this might be a) determining how diverse a metagenome is compared to e.g., a bacterial genome for assembly, b) to compare k-mer diversity among multiple metagenomes, c) exploring the impacts of choice of length k for assembly.




In [1]:
#Put the code here for kmer counting

### Getting a sequence coverage profile:  What genes are present in my metagenome?


Most metagenomic analysis require one to estimate the abundance of reference genes (e.g., orginating from genomes or one's own metagenomic assembly).  This tutorial will cover both cases where references are available or unavailable (requiring de novo assembly).  

### Case I - Reference genomes available.  

For the HMP mock metagenome, the HMP has sequenced the genomes of the isolates used for this simulated dataset.  The list of these genomes can be obtained on the HMP website, and we provide it here in a Github repository, a tool used for collaboratively sharing data and code.  The command below will download data for this tutorial.

In [4]:
!git clone https://github.com/germs-lab/frontiers-review-2015

fatal: destination path 'frontiers-review-2015' already exists and is not an empty directory.


Note:  If the above code has previously been executed, this command will result in an error.  

Next, we will download available reference genomes associated with this metagenome.  The list of the genomes can be seen with the following command.



In [ ]:
#!cat ...

The following command downloads all the data and scripts into a directory called "frontiers-review-2015."  The next step is that we will use a script ("fetch-genomes-fasta.py") to download a list of genomes (listed as separate lines in "ncbi_acc.txt") into a directory named "genomes".

In [6]:
!python frontiers-review-2015/fetch-genomes-fasta.py frontiers-review-2015/ncbi_acc.txt genomes

Fetching NC_005008.1...Done
Fetching NC_005007.1...Done
Fetching NC_005003.1...Done
Fetching NC_005006.1...Done
Fetching NC_005004.1...Done
Fetching NC_009084.1...Done
Fetching NC_005005.1...Done
Fetching NC_000958.1...Done
Fetching NC_000959.1...Done
Fetching NC_009083.1...Done
Fetching NC_001264.1...Done
Fetching NC_001263.1...Done
Fetching NC_004461.1...Done
Fetching NC_009008.1...Done
Fetching NC_010079.1...Done
Fetching NC_007490.1...Done
Fetching NC_009007.1...Done
Fetching NC_007489.1...Done
Fetching NC_004350.1...Done
Fetching NC_007488.1...Done
Fetching NC_007493.1...Done
Fetching NC_007494.1...Done
Fetching NC_009085.1...Done
Fetching NC_009515.1...Done
Fetching NC_009614.1...Done
Fetching NC_000915.1...Done
Fetching NC_003028.3...Done
Fetching NC_000913.2...Done
Fetching NC_006085.1...Done
Fetching NC_003112.2...Done


### Estimating presence of metagenome in assembled contigs

To estimate the representation of reference genes or genomes in your metagenome, you can align reads to references using read mapping software (e.g., Bowtie2, BWA, etc.).  In this tutorial, we will use Bowtie2 which we have previously installed on this server.  We will demonstrate mapping the metagenome to a single reference genome in this command. 

In [ ]:
!bash bowtie.sh REF READS

In [ ]:
!bash bowtie.sh REF READS


### Case II - *De novo* assembly of reference genes.


###  Assembly of the HMP mock metagenome

Assembly is the process of merging overlapping metagenomic reads from *hopefully* the same genome into a longer, continguous sequence (most commonly called a contig).  It is advantageous is that it provides longer lengths for analyzing what the sequence represents, reduces the dataset size for analysis, and provides references that are not dependent on previous knowledge.  

The choice of what assembler to use is not an easy one and is a subject of debate (see http://assemblathon.org).  It is most important to remember that an assembly is a *hypothesized* consensus representation of your dataset.  The assembly itself is an initial step that needs to be followed by an evaluation of its accuracy and usefulness.  For most assemblers, you will need only two things:  Metagenomic reads which you are comfortable with their quality (e.g., quality-score trimmed, adapters removed) and assembly software.  For this tutorial, we will be completing the assembly with an assembler published in 2014 called Megahit (MEGAHIT: An ultra-fast single-node solution for large and complex metagenomics assembly via succinct de Bruijn graph, https://github.com/voutcn/megahit).  Sharpton's review (xxxx) also reviews quite nicely some of the many assembly programs and approaches for metagenomic assembly.  

To reduce the memory that is needed, it is often advantageeous to normalize the distribution of k-mers in a metagenome.  Removing extraneous information not needed for assembly also removes reads that may contain errors and may improve assembly (cite diginorm paper).    

In [ ]:
{MEGAHIT} -m 1e9 -l 250 --k-max 81 -r ecoli_ref-1m.fastq.gz --cpu-only -o megahit_1m

### Estimating abundances of contigs

Once the assembly is finished, you have a set of reference contigs which you can now estimate the abundance of the metagenome.  The approach for doing so is identical to that shown above where you use reference genomes.

In [ ]:
!bash bowtie.sh REF READS

### Annotating the assembled contigs

Determining "who" and/or "what" your contig usually depends on having some idea of the genes you are looking for (e.g., some sort of references).  In our case, we know that the HMP mock community should orignate from a set of genomes (which we actually downlaoded above).  One of the most popular tools of comparing an unknown sequence to a known reference is The Basic Local Alignment Search Tool (or BLAST).  To identify the origin of our contigs, we will align assembled contigs to the genomes used in the HMP mock community.

The first thing we will do is make a searchable database for the BLAST software.

In [ ]:
!formatdb -i ...

Next, we will perform the alignment of the query sequences (our assembled contigs) to the database (the HMP mock genomes).  Additionally, we will also ask BLAST to display the output as a tabular format (-m 8), only give us the best hit (-b 1), and require a minimum E-value threshold (e-value < 1e-5).  You can see other parameters for the program below.

In [ ]:
!blastall -h